In [31]:
import pandas as pd
import numpy
import json


file_mode = "../data/ACTU_STOPS_0.csv"
file_trips = "../data/gtfs3Sept/trips.csv"
file_routes = "../data/gtfs3Sept/routes.csv"
file_stop_times = "../data/gtfs3Sept/stop_times.csv"
file_stops = "../data/gtfs3Sept/stops.csv"
file_calendar = "../data/gtfs3Sept/calendar.csv"

In [32]:
# df_stops = pd.read_csv(file_stops)

In [33]:
"""
creation of the metro trips dataset
"""
df_mode = pd.read_csv(file_mode)
df_trips = pd.read_csv(file_trips)
df_routes = pd.read_csv(file_routes)

df_mode_metro = df_mode[df_mode["mode"]=="M"]
df_metro_id_str= df_mode_metro.numero_lig.astype(str)
df_routes_metro = df_routes[df_routes.route_short_name.isin(df_metro_id_str)]

df_metro_trips = df_trips[df_trips.route_id.isin(df_routes_metro.route_id)]
df_metro_trips = df_metro_trips.drop(["trip_headsign", "block_id", "shape_id"], axis=1)

print(df_metro_trips)

       route_id  service_id             trip_id  direction_id
80299         1   234578052  112585711234578052             1
80300         1   234578052  112585712234578052             1
80301         1   234578052  112585714234578052             0
80302         1   234578052  112585715234578052             0
80303         1   234578052  112585717234578052             1
...         ...         ...                 ...           ...
88833         1   237269601  113423843237269601             1
88834         1   237269601  113423845237269601             0
88835         1   237269601  113423847237269601             1
88836         1   237269601  113423849237269601             0
88837         1   237269601  113423851237269601             1

[8539 rows x 4 columns]


In [34]:
"""
filtering the calendar dataset in week, saturday, sunday
"""
df_calendar = pd.read_csv(file_calendar)
df_calendar["week"] = df_calendar['monday']*df_calendar['tuesday']*df_calendar['wednesday']*df_calendar['thursday']*df_calendar['friday']
df_calendar.drop(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'start_date', 'end_date'], inplace=True, axis=1)

print(df_calendar)

     service_id  saturday  sunday  week
0     234578052         0       0     1
1     236986502         1       0     0
2     236988602         0       1     0
3     236270000         0       0     1
4     236269500         1       0     0
..          ...       ...     ...   ...
432   237102501         1       0     0
433   237472602         0       1     0
434   237451600         0       1     0
435   238036502         1       0     0
436   238255501         1       0     0

[437 rows x 4 columns]


In [35]:
"""
joining calendar and metro trip
"""
df_metro_trip_calendar = pd.merge(df_metro_trips, df_calendar, on="service_id")
print(df_metro_trip_calendar)

      route_id  service_id             trip_id  direction_id  saturday  \
0            1   234578052  112585711234578052             1         0   
1            1   234578052  112585712234578052             1         0   
2            1   234578052  112585714234578052             0         0   
3            1   234578052  112585715234578052             0         0   
4            1   234578052  112585717234578052             1         0   
...        ...         ...                 ...           ...       ...   
8534         1   237269601  113423843237269601             1         0   
8535         1   237269601  113423845237269601             0         0   
8536         1   237269601  113423847237269601             1         0   
8537         1   237269601  113423849237269601             0         0   
8538         1   237269601  113423851237269601             1         0   

      sunday  week  
0          0     1  
1          0     1  
2          0     1  
3          0     1  
4     

In [36]:
"""
merging stop_times and metro_trip_calendar
"""
df_stop_times = pd.read_csv(file_stop_times)
df_stop_times.drop(["pickup_type", "drop_off_type"], inplace=True, axis=1)
df_metro_time = pd.merge(df_metro_trip_calendar, df_stop_times, on="trip_id")
print(df_metro_time )


        route_id  service_id             trip_id  direction_id  saturday  \
0              1   234578052  112585711234578052             1         0   
1              1   234578052  112585711234578052             1         0   
2              1   234578052  112585711234578052             1         0   
3              1   234578052  112585711234578052             1         0   
4              1   234578052  112585711234578052             1         0   
...          ...         ...                 ...           ...       ...   
199764         1   237269601  113423851237269601             1         0   
199765         1   237269601  113423851237269601             1         0   
199766         1   237269601  113423851237269601             1         0   
199767         1   237269601  113423851237269601             1         0   
199768         1   237269601  113423851237269601             1         0   

        sunday  week arrival_time departure_time stop_id  stop_sequence  
0            

In [37]:
print(df_metro_time.stop_id.unique())
print(len(df_metro_time.stop_id.unique()))

['8743' '8731' '8721' '8711' '8701' '8691' '8681' '8671' '8661' '8651'
 '8642' '8641' '8652' '8662' '8672' '8682' '8692' '8702' '8712' '8722'
 '8732' '8742' '8292' '8282' '8272' '8012' '8022' '8032' '8042' '8052'
 '8062' '8072' '8202' '8212' '8222' '8232' '8242' '8252' '8262' '8231'
 '8221' '8211' '8201' '8071' '8061' '8051' '8041' '8031' '8021' '8011'
 '8271' '8281' '8291' '8741' '8251' '8241' '8261' '8161' '8151' '8141'
 '8131' '8121' '8111' '8101' '8091' '8081' '8162' '8733' '8082' '8092'
 '8102' '8112' '8122' '8132' '8142' '8152' '8471' '8461' '8441' '8431'
 '8421' '8411' '8401' '8301' '8311' '8321' '8331' '8341' '8351' '8361'
 '8371' '8381' '8753' '8763' '8773' '8783' '8793' '8803' '8813' '8823'
 '8834' '8833' '8824' '8814' '8804' '8794' '8784' '8774' '8764' '8754'
 '8744' '8382' '8372' '8362' '8352' '8342' '8332' '8322' '8312' '8302'
 '8402' '8412' '8422' '8432' '8442' '8462' '8472']
127


In [57]:

def time_to_sec(time:str):
    time = time.split(":")
    hours, minutes, seconds = int(time[0]), int(time[1]), int(time[2])
    return hours*60*60+minutes*60+seconds


temp = []
for i in range(len(df_metro_time)):
    if df_metro_time.stop_id[i] == "8743" and df_metro_time.direction_id[i] == 1 and df_metro_time.week[i] == 1:
        temp.append(df_metro_time.arrival_time[i])
temp.sort()
print(temp)

['05:09:23', '05:25:27', '05:29:00', '05:33:36', '05:35:00', '05:37:36', '05:37:41', '05:41:28', '05:46:04', '05:47:20', '05:53:36', '05:53:36', '05:58:37', '06:00:01', '06:03:39', '06:09:01', '06:09:59', '06:16:19', '06:19:01', '06:22:39', '06:23:24', '06:28:24', '06:28:50', '06:31:54', '06:32:18', '06:34:48', '06:35:04', '06:37:18', '06:41:24', '06:41:34', '06:44:04', '06:44:34', '06:47:44', '06:49:09', '06:50:54', '06:54:04', '06:54:04', '06:57:14', '06:57:14', '07:00:24', '07:00:24', '07:03:34', '07:03:34', '07:06:44', '07:06:44', '07:09:54', '07:09:54', '07:13:04', '07:13:04', '07:16:14', '07:16:14', '07:19:24', '07:19:24', '07:22:34', '07:22:34', '07:25:44', '07:25:44', '07:28:54', '07:28:54', '07:32:04', '07:32:04', '07:35:14', '07:35:14', '07:38:24', '07:38:24', '07:41:34', '07:41:34', '07:44:44', '07:44:44', '07:47:54', '07:47:54', '07:51:04', '07:51:04', '07:54:14', '07:54:14', '07:57:24', '07:57:24', '08:00:34', '08:00:34', '08:03:44', '08:03:44', '08:06:54', '08:06:54', '08

In [49]:
import datetime